### Intact Forest data manipulation

With these notebook, we can reproduce intact forest data as binary system. Pixel values are going to be 1 or 0. For this process, 
firstly, we create 512*512 tile from raw data, then we get data that over land area.(delete files that over sea) Then we change nodata value(-32768) to 0

In [1]:
import geopandas as gpd
from osgeo import gdal, ogr, osr
import os
import glob
import numpy

In [ ]:
raw_countries=gpd.read_file('fao/shp_from_un/BNDA25_CTY.shp')

In [116]:
tifs_path=glob.glob('intact_forest/*tif')

In [169]:
tifs_path

['intact_forest/intact_forest_landscapes_2013.tif',
 'intact_forest/intact_forest_landscapes_2000.tif',
 'intact_forest/intact_forest_landscapes_2016.tif']

In [170]:
# we get image one by one so we use index number of list
%%time
for img in tifs_path[1:2]:
    # define tiles path
    path='intact_forest/tiles_2000'
    input_path=img
   
    # name of tile image
    output_path=path+'/intact_forest_tile_'
    ds = gdal.Open(img)
    band = ds.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize
    # define your tile size
    # it could be 256,512 or 1024. it depends on your scope
    tile_size_x = 512
    tile_size_y = 512
    for i in range(0, xsize, tile_size_x):
        if xsize-i < tile_size_x:
            i=xsize-tile_size_x            
        for j in range(0, ysize, tile_size_y):
            if ysize-j < tile_size_y:
                j=ysize-tile_size_y
    
            com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(input_path)+ " " + str(output_path)  + str(i) + "_" + str(j) + ".tif"
            #print(com_string)
            os.system(com_string)

CPU times: user 190 ms, sys: 203 ms, total: 393 ms
Wall time: 2min 14s


In [171]:
#get all tiles path
tiles_path=glob.glob('intact_forest/tiles_2000/*.tif')

txtfile='tileindex.txt'

In [173]:
# we create tile index to intersect with world shp file
for i in tiles_path:

    with open(txtfile, 'a') as the_file:

        the_file.write(i+"\n")

output_name='intact_forest/tile_index_shp/2000.shp'

cmd=f'gdaltindex -tileindex location -t_srs EPSG:4326 -f "ESRI Shapefile" {output_name} --optfile {txtfile}'

os.system(cmd)

os.remove(txtfile)

In [174]:
#read tile index file

tile=gpd.read_file('intact_forest/tile_index_shp/2000.shp')

In [176]:
#intersect with world shp file
inter=gpd.overlay(tile, raw_countries, how='intersection')

In [193]:
inter.head(2)

,location,ISO3CD,ROMNAM,MAPLAB,CONTCD,MAPCLR,STSCOD,Shape_Leng,Shape_Area,geometry
0,intact_forest/tiles_2000/intact_forest_tile_35...,CHN,China,China,ASI,CHN,1,304.899042,948.513659,"POLYGON ((125.95743 52.73381, 124.76886 52.733..."
1,intact_forest/tiles_2000/intact_forest_tile_33...,CHN,China,China,ASI,CHN,1,304.899042,948.513659,"MULTIPOLYGON (((116.26966 45.76242, 116.26966 ..."


In [178]:
#to get the name of the files which our target images, we just get location column
intersect_list=inter['location']

In [127]:
#we can export data as shp file but it's not necessary now
#inter.to_file('intact_forest/interect_shp/2000.shp',driver='ESRI Shapefile')

In [181]:
#create list of the name
tif_basename=[os.path.basename(i) for i in intersect_list]
len(tif_basename)

2541

In [182]:
# we eliminate the duplicate name with 'set'
tif_basename=list(set(tif_basename))

In [183]:
len(tif_basename)

1347

In [184]:
#if name is not in basename list, delete the file
for i in tiles_path:
    if os.path.basename(i) in tif_basename :
        continue
    else:
        os.remove(i)

#### Change the nodata

In [190]:
tiles=glob.glob('intact_forest/tiles_2016/*.tif')
len(tiles)

1261

In [191]:
tiles[0]

'intact_forest/tiles_2016/intact_forest_tile_8704_3584.tif'

In [192]:
for img in tiles:
    hdf_ds = gdal.Open(img, gdal.GA_ReadOnly)
    array_img=hdf_ds.ReadAsArray()
    # change the nodata -32768 >>>>> 0
    np_where_img = numpy.where((array_img==-32768),0,(array_img))
    #####################################
    s_srs = hdf_ds.GetProjectionRef()
    osng = osr.SpatialReference ()
    osng.SetFromUserInput (s_srs)
    geo_t = hdf_ds.GetGeoTransform ()
    x_size = hdf_ds.RasterXSize # Raster xsize
    y_size = hdf_ds.RasterYSize # Raster ysize
    #create gdal memory file
    mem_drv = gdal.GetDriverByName( 'MEM' )
    dest = mem_drv.Create('', x_size,y_size, 1, gdal.GDT_Int16)
    dest.SetGeoTransform( geo_t )
    dest.SetProjection ( osng.ExportToWkt() )
    #dest.GetRasterBand(1).SetNoDataValue(-32768)
    dest.GetRasterBand(1).WriteArray(np_where_img)
    name=os.path.basename(img) 
    output_path='intact_forest/tiles_2016_recalc/'+name
    gdal.Warp(output_path, dest, format = 'GTiff')
    dest=None
    mem_drv=None
    np_where_img=None